In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("D:\Automation\Data Cleaning\Datasets\walmart\walmart.csv")
print(df.head())

   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  05-02-2010    1643690.90             0        42.31       2.572   
1      1  12-02-2010    1641957.44             1        38.51       2.548   
2      1  19-02-2010    1611968.17             0        39.93       2.514   
3      1  26-02-2010    1409727.59             0        46.63       2.561   
4      1  05-03-2010    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB
None


In [7]:
print(df.shape)

(6435, 8)


In [8]:
print(df.describe(include='all'))

              Store        Date  Weekly_Sales  Holiday_Flag  Temperature  \
count   6435.000000        6435  6.435000e+03   6435.000000  6435.000000   
unique          NaN         143           NaN           NaN          NaN   
top             NaN  05-02-2010           NaN           NaN          NaN   
freq            NaN          45           NaN           NaN          NaN   
mean      23.000000         NaN  1.046965e+06      0.069930    60.663782   
std       12.988182         NaN  5.643666e+05      0.255049    18.444933   
min        1.000000         NaN  2.099862e+05      0.000000    -2.060000   
25%       12.000000         NaN  5.533501e+05      0.000000    47.460000   
50%       23.000000         NaN  9.607460e+05      0.000000    62.670000   
75%       34.000000         NaN  1.420159e+06      0.000000    74.940000   
max       45.000000         NaN  3.818686e+06      1.000000   100.140000   

         Fuel_Price          CPI  Unemployment  
count   6435.000000  6435.000000   643

In [9]:
print(df.dtypes)

Store             int64
Date             object
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object


In [10]:
# 2 Missing Data Assessment

In [12]:
df.isnull().sum()

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [20]:
empty_count = (df['Date'] == '').sum()
print(f"Empty strings {empty_count}")
unknown_count = df['Date'].str.contains('unknown', case=False, na=False).sum()
print(f"Unknown strings {unknown_count}")

Empty strings 0
Unknown strings 0


In [33]:
df_clean = df.copy()

In [34]:
def parse_date(date_str):
    try:
        for fmt in ['%d-%m-%Y', '%d/%m/%Y', '%d.%m.%Y']:
            try:
                return pd.to_datetime(date_str, format=fmt)
            except:
                return np.nan
    except:
        return np.nan

In [35]:
df_clean['Date'] = df_clean['Date'].apply(parse_date)

In [38]:
df_clean.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


In [42]:
df_clean.dtypes

Store                    int64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

In [43]:
#IQR

In [51]:
def iqr(col):
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    number_of_outofbound_rows = (df[col] > upper_bound).sum()
    return f"{col} IQR bounds: [{lower_bound:.0f}, {upper_bound:.0f}, {number_of_outofbound_rows}]"

In [52]:
#Identify IQR for all the numerical columns

In [53]:
for col in df_clean.select_dtypes(include=['int64','float64']).columns:
    print(iqr(col))

Store IQR bounds: [-21, 67, 0]
Weekly_Sales IQR bounds: [-746863, 2720371, 34]
Holiday_Flag IQR bounds: [0, 0, 450]
Temperature IQR bounds: [6, 116, 0]
Fuel_Price IQR bounds: [2, 5, 0]
CPI IQR bounds: [10, 334, 0]
Unemployment IQR bounds: [4, 11, 378]


In [63]:
df_clean['Holiday_Flag'].astype('boolean')

0       False
1        True
2       False
3       False
4       False
        ...  
6430    False
6431    False
6432    False
6433    False
6434    False
Name: Holiday_Flag, Length: 6435, dtype: boolean

In [67]:
df_clean.to_csv('D:\Automation\Data Cleaning\Datasets\walmart\cleaned_walmart.csv', index=False)

In [65]:
df_clean.dtypes

Store                    int64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object